In [51]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_boston

In [52]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,8 hours 39 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_root_3dql4z
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,447.8 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [ ]:
## LOAD DADASET
boston_dataset = load_boston()
boston_data = boston_dataset['data']                  # NDARRAY
boston_feature_names = boston_dataset['feature_names']
boston_df = pd.DataFrame(data=boston_data, columns=boston_feature_names)
boston_df['price']= boston_dataset['target']


train_x, test_x, train_y, test_y = train_test_split(boston_df[boston_dataset['feature_names']], boston_df['price'], random_state=0, test_size=0.33)



train_df = train_x.copy()
train_df['price']=train_y


test_df = test_x.copy()
test_df['price']=test_y

In [ ]:
## H2O FRAME으로 변환
train_hf = h2o.H2OFrame(train_df)
test_hf = h2o.H2OFrame(test_df)
target_col = 'price'
feature_col = hf.columns.remove(target_col)


In [53]:
## H2O AUTOML 객체 생성

aml = H2OAutoML(max_models=10,
                nfolds=4,
               exclude_algos=['DeepLearning'],
               keep_cross_validation_predictions=True,
               seed=5)

In [54]:
## 학습진행
aml.train(x=feature_col,
         y=target_col,
         training_frame=train_hf,
         leaderboard_frame=test_hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [55]:
print(aml.leaderboard)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_3_AutoML_20200912_165208,16.3214,4.03997,16.3214,2.4378,0.15653
StackedEnsemble_AllModels_AutoML_20200912_165208,16.7502,4.09271,16.7502,2.43939,0.161474
GBM_1_AutoML_20200912_165208,17.0834,4.13321,17.0834,2.56783,0.173204
GBM_4_AutoML_20200912_165208,17.5063,4.18406,17.5063,2.50014,0.164305
StackedEnsemble_BestOfFamily_AutoML_20200912_165208,17.6023,4.19551,17.6023,2.4634,0.164984
XGBoost_3_AutoML_20200912_165208,17.9895,4.2414,17.9895,2.64339,0.1728
XGBoost_2_AutoML_20200912_165208,18.1644,4.26197,18.1644,2.56878,0.166736
GBM_2_AutoML_20200912_165208,18.2627,4.27349,18.2627,2.6272,0.16471
XGBoost_1_AutoML_20200912_165208,19.3741,4.40161,19.3741,2.62792,0.181298
DRF_1_AutoML_20200912_165208,19.9577,4.4674,19.9577,2.595,0.179014


In [56]:
# 동일기준으로 검증

from sklearn.metrics import mean_squared_error

h_pred = aml.predict(test_hf).as_data_frame()

mean_squared_error(test_y, h_pred, squared=False)

gbm prediction progress: |████████████████████████████████████████████████| 100%


4.039971992143172

In [57]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_20200912_165208


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,67.0,67.0,18804.0,7.0,8.0,7.910448,11.0,23.0,17.74627




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1.6726722087239496
RMSE: 1.2933182936632226
MAE: 0.8796531556278555
RMSLE: 0.06062006757147889
Mean Residual Deviance: 1.6726722087239496

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 11.291451301588365
RMSE: 3.360275480014751
MAE: 2.312870552040231
RMSLE: 0.14141817055146663
Mean Residual Deviance: 11.291451301588365

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid
0,mae,2.3140101,0.37466517,1.904815,2.5537097,2.097211,2.7003045
1,mean_residual_deviance,11.301816,3.776805,8.97746,14.195428,7.2188263,14.815551
2,mse,11.301816,3.776805,8.97746,14.195428,7.2188263,14.815551
3,r2,0.8669158,0.04141888,0.914606,0.82987833,0.8884568,0.83472216
4,residual_deviance,11.301816,3.776805,8.97746,14.195428,7.2188263,14.815551
5,rmse,3.324952,0.5733072,2.9962409,3.767682,2.6867874,3.8490975
6,rmsle,0.13919966,0.029218858,0.11839506,0.16713324,0.10991846,0.16135184



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2020-09-12 16:52:17,0.364 sec,0.0,9.275733,6.755825,86.039220
1,,2020-09-12 16:52:17,0.384 sec,5.0,6.085350,4.360604,37.031484
2,,2020-09-12 16:52:17,0.398 sec,10.0,4.313417,3.015271,18.605567
3,,2020-09-12 16:52:17,0.407 sec,15.0,3.283732,2.254147,10.782898
4,,2020-09-12 16:52:17,0.417 sec,20.0,2.642573,1.788547,6.983193
5,,2020-09-12 16:52:17,0.428 sec,25.0,2.273683,1.516020,5.169636
6,,2020-09-12 16:52:17,0.437 sec,30.0,2.092696,1.371596,4.379377
7,,2020-09-12 16:52:17,0.447 sec,35.0,1.878529,1.241547,3.528870
8,,2020-09-12 16:52:17,0.458 sec,40.0,1.729937,1.151740,2.992682
9,,2020-09-12 16:52:17,0.475 sec,45.0,1.627150,1.091694,2.647616



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,LSTAT,62082.183594,1.000000,0.445224
1,RM,51464.589844,0.828975,0.369080
2,DIS,5941.545410,0.095705,0.042610
3,INDUS,4901.024902,0.078944,0.035148
4,CRIM,3698.578369,0.059576,0.026524
5,PTRATIO,3354.606201,0.054035,0.024058
6,NOX,2688.640869,0.043308,0.019282
7,AGE,1755.909302,0.028284,0.012593
8,TAX,1385.072144,0.022310,0.009933
9,B,1268.046265,0.020425,0.009094
